In [5]:
import sys
sys.path.insert(0, "..")
from utils import linalg
from importlib import reload
import distributed
from distributed import kernel
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import matmul
import numpy as np
import pywren
from utils import misc
import math
import concurrent.futures as fs
import time
import os
from numba import jit
import scipy.linalg
from sklearn import metrics
from scipy.linalg import LinAlgError

from distributed import bcd
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [6]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [23]:
X_EXISTS = True
K_EXISTS = True
NORMS_EXIST = True
GAMMA = 1e1
NUM_TRAIN = 1281167

In [5]:
pwex = pywren.default_executor(job_max_runtime=600)

In [4]:
%time X_block_sharded = ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017",\
                                shard_size_0=4096,\
                                bucket="vaishaalpywrenlinalg",\
                                n_jobs=32)

CPU times: user 120 ms, sys: 12 ms, total: 132 ms
Wall time: 203 ms


In [27]:
key = "rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_blocks_no_normalize_04.22.2017, {0})".format(GAMMA)

if (not K_EXISTS):
   K = kernel.compute_kernel_pywren(pwex, key, X_block_sharded, 4, gamma=GAMMA, num_jobs=4500, num_features=X_block_sharded.shape[1])
else: 
    K = ShardedSymmetricMatrix(key, bucket="vaishaalpywrenlinalg")

In [28]:
K.get_block(312,312)

array([[  9.99997616e-01,   1.39793837e-02,   1.28975476e-03, ...,
          4.61571617e-03,   1.12510612e-03,   1.91534622e-04],
       [  1.39793837e-02,   9.99999404e-01,   6.86560431e-03, ...,
          5.75694479e-02,   2.42453930e-03,   1.98401650e-03],
       [  1.28975476e-03,   6.86560431e-03,   1.00000060e+00, ...,
          4.17747200e-02,   6.69077411e-02,   2.61596274e-02],
       ..., 
       [  4.61571617e-03,   5.75694479e-02,   4.17747088e-02, ...,
          9.99999404e-01,   3.64136742e-03,   7.12921983e-03],
       [  1.12510507e-03,   2.42454256e-03,   6.69077113e-02, ...,
          3.64136742e-03,   1.00000000e+00,   4.58960570e-02],
       [  1.91534622e-04,   1.98401650e-03,   2.61596274e-02, ...,
          7.12921983e-03,   4.58961241e-02,   1.00000000e+00]], dtype=float32)

In [13]:
idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")
labelsTrain = np.load("/home/ubuntu/imagenet-data/imagenet_train_labels.npy")[idxs]

In [14]:
labelsTrain

array([ 955.,  978.,   98., ...,  752.,   91.,  239.])

In [15]:
y_train = np.eye(1000)[labelsTrain.astype('int')]

In [16]:
K.get_block(0,0)

array([[ 1.00000012,  0.58216029,  0.85013938, ...,  0.42229491,
         0.72843969,  0.63366985],
       [ 0.58216029,  1.        ,  0.60700768, ...,  0.45322865,
         0.64749706,  0.83554441],
       [ 0.85013938,  0.60700768,  1.        , ...,  0.44800186,
         0.79409033,  0.69017005],
       ..., 
       [ 0.42229491,  0.45322865,  0.44800186, ...,  0.99999928,
         0.41453877,  0.38948178],
       [ 0.72843969,  0.64749706,  0.79409033, ...,  0.41453877,
         1.00000012,  0.71894205],
       [ 0.63366985,  0.83554441,  0.69017005, ...,  0.38948175,
         0.71894205,  1.        ]], dtype=float32)

In [29]:
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [ ]:
sys.stdout = terminal_stdout
reload(bcd)
X_1_epoch = bcd.block_kernel_solve(K, y_train, epochs=2, lambdav=1e-1, blocks_per_iter=4)

In [45]:
metrics.accuracy_score(np.argmax(X_1_epoch[1], axis=1), labelsTrain)

0.18821121680467887

In [22]:
model = np.load("/dev/shm/model_iter_70_train_acc_0.809784360664925_next_block_acc_0.137939453125_norm_725.7616952103908.npy")

In [164]:
svd_stuff = np.linalg.svd(X_1_epoch)

In [172]:
svd_stuff[1][0]/svd_stuff[1][-1]

10493230837679216.0

In [177]:
np.diag(X_1_epoch)

memmap([ 0.38845128,  0.89646912,  0.44712415, ...,  0.26498708,
        0.56446284,  0.66869539])

In [179]:
K_block = K.get_block(0,0)

In [187]:
np.diag(K_block)

array([ 0.99999881,  0.99999988,  0.99999988, ...,  1.        ,
        0.99999952,  1.        ], dtype=float32)

In [183]:
svd_stuff[1]

array([  2.17837646e+03,   3.11883759e+02,   1.85900482e+02, ...,
         2.88411011e-05,   2.49138957e-05,   1.49347890e-17], dtype=float32)

In [42]:
x_start, y_hat =  X_1_epoch

In [145]:
%time a = bcd.fast_kernel_column_block_async(K, [0,1,2])
%time fs.wait(a)

[0, 1, 2]
CPU times: user 276 ms, sys: 1.78 s, total: 2.06 s
Wall time: 2.04 s
CPU times: user 912 ms, sys: 84 ms, total: 996 ms
Wall time: 1min 45s


DoneAndNotDoneFutures(done={<Future at 0x7f2d00221898 state=finished returned tuple>, <Future at 0x7f2ce04cf898 state=finished returned tuple>, <Future at 0x7f2ce04cf0b8 state=finished returned tuple>, <Future at 0x7f2de057b8d0 state=finished returned tuple>, <Future at 0x7f2ce04cf9e8 state=finished returned tuple>, <Future at 0x7f1f5df06a20 state=finished returned tuple>, <Future at 0x7f2ce04cfa58 state=finished returned tuple>, <Future at 0x7f2d00221a58 state=finished returned tuple>, <Future at 0x7f1f5d3a72b0 state=finished returned tuple>, <Future at 0x7f2cc2186b70 state=finished returned tuple>, <Future at 0x7f2dc0295470 state=finished returned tuple>, <Future at 0x7f2de057bc88 state=finished returned tuple>, <Future at 0x7f2dc02954a8 state=finished returned tuple>, <Future at 0x7f2ccba2e4e0 state=finished returned tuple>, <Future at 0x7f2cc993ad30 state=finished returned tuple>, <Future at 0x7f38c1699d68 state=finished returned tuple>, <Future at 0x7f2ce04cf588 state=finished ret

In [146]:
k = bcd.load_mmap(*a[0].result())

In [152]:
bcd.block_kernel_solve(K, 

memmap([[ 0.99999881,  0.40342984,  0.36615327, ...,  0.30506128,
         0.67470342,  0.40624243],
       [ 0.40342984,  0.99999988,  0.64286745, ...,  0.64839852,
         0.69248831,  0.80197585],
       [ 0.36615327,  0.64286745,  0.99999988, ...,  0.68896604,
         0.72726387,  0.8055684 ],
       ..., 
       [ 0.33696416,  0.68765193,  0.73388755, ...,  0.874237  ,
         0.59649545,  0.85790408],
       [ 0.35511884,  0.82535362,  0.73779732, ...,  0.74089462,
         0.64481544,  0.86099726],
       [ 0.28665397,  0.70327604,  0.67107129, ...,  0.90301561,
         0.5732618 ,  0.85399121]])

In [38]:
k

NameError: name 'k' is not defined